In [1]:
from ml_extra.decorators.mlflow.exp_tracking import mlflow_experiment 
from ml_extra.decorators.mlflow.exp_tracking import mlflow_client
from ml_extra.decorators.mlflow.exp_tracking import mlflow_tracking_uri
from deckgen.client.openai_client import OpenAIClient
from dotenv import load_dotenv
from prompteng.prompts import templates
from prompteng.prompts.parser import QAParser
from prompteng.utils.filesystem import get_root_path

import os 
import mlflow

In [2]:
root_path = get_root_path()
os.environ["MLFLOW_TRACKING_URI"] = (root_path / "mlruns").as_uri()

In [3]:
@mlflow_tracking_uri
@mlflow_experiment(name="simple_prompt_experiment")
def log_simple_prompt(**kwargs):
    """
    A simple function to demonstrate logging a prompt in an MLflow experiment.
    """

    template = kwargs.get("template")
    name = kwargs.get("name", "simple_prompt")
    with mlflow.start_run(run_name="simple_prompt_run"):
        mlflow.genai.register_prompt(
            name=name,
            template=template,
        )

In [4]:
# log_simple_prompt(template= templates.QUESTION_ASKER, name ="question_asker_prompt")
# log_simple_prompt(template=templates.TOPIC_FINDER, name="topic_finder_prompt")
log_simple_prompt(template=templates.QUESTION_ASKING, name="question_asking_prompt")

Setting MLflow tracking URI...
Setting MLflow experiment: simple_prompt_experiment


In [5]:
loaded_template = mlflow.genai.load_prompt(name_or_uri ="topic_finder_prompt", version="4")
loaded_template_qa = mlflow.genai.load_prompt(name_or_uri="question_asking_prompt", version="5")
print(loaded_template_qa)

PromptVersion(name=question_asking_prompt, version=5, template=
You are an expert in {{expert...)


In [6]:
load_dotenv(dotenv_path = root_path / ".env")

True

In [7]:
client = OpenAIClient()

In [8]:
import json

In [9]:
text = """ 
Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processing, 
scientific visualization, algorithms and systems to extract or extrapolate knowledge from potentially noisy, structured, or unstructured data.

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).
Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.
Data science is "a concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena"
with data. It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science,
and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gray imagined data science as a "fourth paradigm" of science (empirical, theoretical, computational, and now data-driven) and asserted that "everything about science is changing because of the impact of information technology" and the data deluge.[7][8]

A data scientist is a professional who creates programming code and combines it with statistical knowledge to summarize data.
"""
text =   """ 
Discover Azure Functions

Azure Functions is a serverless solution that allows you to write less code, maintain less infrastructure, and save on costs. Instead of worrying about deploying 
and maintaining servers, the cloud infrastructure provides all the up-to-date resources needed to keep your applications running.
We often build systems to react to a series of critical events. Whether you're building a web API, responding to database changes,
processing IoT data streams, or even managing message queues - every application needs a way to run some code as these events occur.

Azure Functions supports triggers, which are ways to start execution of your code, and bindings, 
which are ways to simplify coding for input and output data. There are other integration and automation
services in Azure and they all can solve integration problems and automate business processes.
They can all define input, actions, conditions, and output.

Compare Azure Functions and Azure Logic Apps
Both Functions and Logic Apps are Azure Services that enable serverless workloads. Azure Functions is a serverless compute service,
whereas Azure Logic Apps is a serverless workflow integration platform. Both can create complex orchestrations. 
An orchestration is a collection of functions or steps, called actions in Logic Apps, that are executed to accomplish a complex task.

For Azure Functions, you develop orchestrations by writing code and using the Durable Functions extension. 
For Logic Apps, you create orchestrations by using a GUI or editing configuration files.
"""


In [10]:
topics = client.request(
    method="POST",
    endpoint="responses",
    data = json.dumps({"model":"gpt-3.5-turbo","input": loaded_template.format(text=text)}))

In [11]:
expertise = " ".join(topics.json()["output"][0]["content"][0]["text"].split("\n"))

In [12]:
response = client.request(
    method="POST",
    endpoint="responses",
    data=json.dumps({
        "model": "gpt-3.5-turbo",
        "input": loaded_template_qa.format(expertise=expertise, text=text)
    })
)

In [13]:
qa_string = response.json()["output"][0]["content"][0]["text"]

In [14]:
qa_parser = QAParser(text=qa_string)
qa_list = qa_parser.parse()


In [15]:
qa_list

[{'question': 'What is Azure Functions and how does it differ from traditional server infrastructure?',
  'answer': 'Azure Functions is a serverless solution that allows you to write less code, maintain less infrastructure, and save on costs by leveraging cloud resources instead of managing servers.'},
 {'question': 'What are triggers and bindings in Azure Functions?',
  'answer': 'Triggers are ways to start the execution of code, while bindings are ways to simplify coding for input and output data in Azure Functions.'},
 {'question': 'How do Azure Functions and Azure Logic Apps compare in terms of their purpose?',
  'answer': 'Azure Functions is a serverless compute service for running functions, while Azure Logic Apps is a serverless workflow integration platform for creating orchestrations.'},
 {'question': 'What is an orchestration in the context of Azure Functions and Logic Apps?',
  'answer': 'An orchestration is a collection of functions or steps (actions in Logic Apps) that are

In [16]:
from deckgen.pipelines.qa_pipeline import QAToolKit

In [18]:
text =   """ 
Discover Azure Functions

Azure Functions is a serverless solution that allows you to write less code, maintain less infrastructure, and save on costs. Instead of worrying about deploying 
and maintaining servers, the cloud infrastructure provides all the up-to-date resources needed to keep your applications running.
We often build systems to react to a series of critical events. Whether you're building a web API, responding to database changes,
processing IoT data streams, or even managing message queues - every application needs a way to run some code as these events occur.

Azure Functions supports triggers, which are ways to start execution of your code, and bindings, 
which are ways to simplify coding for input and output data. There are other integration and automation
services in Azure and they all can solve integration problems and automate business processes.
They can all define input, actions, conditions, and output.

Compare Azure Functions and Azure Logic Apps
Both Functions and Logic Apps are Azure Services that enable serverless workloads. Azure Functions is a serverless compute service,
whereas Azure Logic Apps is a serverless workflow integration platform. Both can create complex orchestrations. 
An orchestration is a collection of functions or steps, called actions in Logic Apps, that are executed to accomplish a complex task.

For Azure Functions, you develop orchestrations by writing code and using the Durable Functions extension. 
For Logic Apps, you create orchestrations by using a GUI or editing configuration files.
"""

qa_toolkit = QAToolKit(input_text=text)
qa_list = qa_toolkit.generate_qa()
for qa in qa_list:
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answer']}")
    print("-" * 40)

Question: What is Azure Functions and how does it differ from traditional server infrastructure?
Answer: Azure Functions is a serverless solution that allows you to write less code, maintain less infrastructure, and save on costs by leveraging cloud resources instead of deploying and maintaining servers.
----------------------------------------
Question: What are triggers and bindings in Azure Functions?
Answer: Triggers in Azure Functions are ways to start the execution of your code based on events, while bindings simplify coding for input and output data.
----------------------------------------
Question: How do Azure Functions and Azure Logic Apps differ in terms of their purpose?
Answer: Azure Functions is a serverless compute service for writing code orchestrations, while Azure Logic Apps is a serverless workflow integration platform that allows creating orchestrations using a GUI or configuration files.
----------------------------------------
Question: What is an orchestration i

In [ ]:
qa_list